Using openai batch processing to run fc metric on a large dataset
This is cheaper and the laptop does not have to be on all the time.

In [3]:
import json

from openai import OpenAI
import os
import pandas as pd
from fc_metric import calculate_fc_using_gpt

In [4]:
if not os.getenv("OPENAI_API_KEY"):
    raise ValueError(
        "Please set the OPENAI_API_KEY environment variable before running the factual consistency metric script"
    )
client = OpenAI()

In [29]:
abstracts_file_name = 'scientific_nature_DOIs'

In [32]:
df = pd.read_csv(f"../data/identifiers_and_abstracts/{abstracts_file_name}.csv")
df.dropna(subset=["sem_scholar_abstract"], inplace=True, ignore_index=True)
df

,id,category,title,url,identifier,sem_scholar_title,sem_scholar_abstract
0,815vqz,scientific,Signal detected from the first stars in the un...,https://www.nature.com/articles/nature25791,DOI:10.1038/nature25791,Possible interaction between baryons and dark-...,The cosmic radio-frequency spectrum is expecte...
1,81ymjh,scientific,Artificial intelligence techniques able to lea...,https://www.nature.com/articles/s41567-018-0048-5,DOI:10.1038/s41567-018-0048-5,Neural-network quantum state tomography,The experimental realization of increasingly c...
2,82bpf1,scientific,Thrombin@Fe 3 O 4 nanoparticles for use as a h...,https://www.nature.com/articles/s41598-017-186...,DOI:10.1038/s41598-017-18665-4,Thrombin@Fe3O4 nanoparticles for use as a hemo...,Bleeding remains one of the main causes of pre...
3,82dzyr,scientific,Scientists have found a breakthrough technique...,https://www.nature.com/articles/s41557-018-0009-8,DOI:10.1038/s41557-018-0009-8,Control over phase separation and nucleation u...,Control over the nucleation of new phases is h...
4,82es30,scientific,Simulating Uneven Trails Provides Insight Into...,https://www.nature.com/articles/s41598-018-210...,DOI:10.1038/s41598-018-21018-4,Challenging human locomotion: stability and mo...,The need to move over uneven terrain is a dail...
...,...,...,...,...,...,...,...
6813,dph7bi,scientific,Neoceroplatus betaryiensisnov. sp. (Diptera: K...,https://www.nature.com/articles/s41598-019-477...,DOI:10.1038/s41598-019-47753-w,Neoceroplatus betaryiensis nov. sp. (Diptera: ...,Blue shining fungus gnats (Diptera) had been l...
6814,dpi8uf,scientific,"In this mouse model of epilepsy, the temperatu...",https://www.nature.com/articles/s41374-019-0335-5,DOI:10.1038/s41374-019-0335-5,Temperature elevation in epileptogenic foci ex...,Physiological brain temperature is an importan...
6815,dppdhj,scientific,"Based on brain scans data from 1,774 people wi...",https://www.nature.com/articles/s41467-019-130...,DOI:10.1038/s41467-019-13005-8,Altered structural brain asymmetry in autism s...,Altered structural brain asymmetry in autism s...
6816,dpstjb,scientific,Researchers demonstrate generation of 2D &amp;...,https://www.nature.com/articles/s41377-019-0201-7,DOI:10.1038/s41377-019-0201-7,Dielectric metasurfaces for complete and indep...,Metasurfaces are optically thin metamaterials ...


In [48]:
df['fc_score'] = -1
if not os.path.exists(f"{abstracts_file_name}_fc_scores.csv"):
    df_out = df.copy()
else:
    df_out = pd.read_csv(f"{abstracts_file_name}_fc_scores.csv")

## Creating a batch request for the factual consistency metric using the lines from min to max

In [52]:
min = 6000
max = 9000
file_name = f"request_abstracts_{abstracts_file_name}_{min}_{max}.jsonl"
with open(file_name, "w+") as f:
    for i, row in df.iterrows():
        if i < min:
            continue
        if i >= max:
            break
        messages = calculate_fc_using_gpt(row["sem_scholar_abstract"], row["title"], row["sem_scholar_title"], False, return_prompt=True)
        input = {
            "custom_id": "row-" + str(i) + "-fc-id-" + str(row["id"]),
            "method": "POST",
            "url": "/v1/chat/completions",
            "body": {
                "model": "gpt-4o-mini",
                "messages": messages
            }
        }
        f.write(json.dumps(input) + "\n")

In [53]:
# Uploading a request file to OpenAI
request_file = client.files.create(
  file=open(file_name, "rb"),
  purpose="batch"
)
request_file

FileObject(id='file-QytrfSTXdqx9KaYLJ3fGLy', bytes=1957229, created_at=1733109049, filename='request_abstracts_scientific_nature_DOIs_6000_9000.jsonl', object='file', purpose='batch', status='processed', status_details=None)

In [66]:
# Creating a batch request using the uploaded input file
batch = client.batches.create(
  input_file_id=request_file.id,
  endpoint="/v1/chat/completions",
  completion_window="24h"
)
batch

Batch(id='batch_674d2b8989e881909b6097794e77c74b', completion_window='24h', created_at=1733110665, endpoint='/v1/chat/completions', input_file_id='file-QytrfSTXdqx9KaYLJ3fGLy', object='batch', status='validating', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=1733197065, failed_at=None, finalizing_at=None, in_progress_at=None, metadata=None, output_file_id=None, request_counts=BatchRequestCounts(completed=0, failed=0, total=0))

## Retrieving the batch to find out about the completion status

In [ ]:
batch

In [74]:
news_batch = 'batch_674cd0728f788190a60e7438d277162a'
science_batch = 'batch_674cdc674f2881909f46ddcedd77eb01'
science_batch_2 = 'batch_674d23e3e0508190981fb757444b08fd'
science_batch_3 = 'batch_674d2b8989e881909b6097794e77c74b'

In [77]:
batch = client.batches.retrieve(science_batch_3)
batch

Batch(id='batch_674d2b8989e881909b6097794e77c74b', completion_window='24h', created_at=1733110665, endpoint='/v1/chat/completions', input_file_id='file-QytrfSTXdqx9KaYLJ3fGLy', object='batch', status='completed', cancelled_at=None, cancelling_at=None, completed_at=1733111217, error_file_id=None, errors=None, expired_at=None, expires_at=1733197065, failed_at=None, finalizing_at=1733111161, in_progress_at=1733110666, metadata=None, output_file_id='file-Q1w2315H2cgVoVJ6KadSUh', request_counts=BatchRequestCounts(completed=818, failed=0, total=818))

## Downloading the output file to get the factual consistency scores and adding them to the output_dataframe

In [78]:
response_content = client.files.content(batch.output_file_id)

In [79]:
for line in response_content.iter_lines():
    output = json.loads(line)
    score_json = output['response']["body"]["choices"][0]["message"]["content"]
    score = json.loads(score_json).get("score", -2)
    custom_id = output["custom_id"]
    row_id = int(custom_id.split("-")[1])
    post_id = custom_id.split("-")[4]
    if df.loc[row_id, "id"] != post_id:
        raise ValueError(f"Row ID {row_id} does not match post ID {post_id}")
    df_out.loc[row_id, 'fc_score'] = score
df_out

,id,category,title,url,identifier,sem_scholar_title,sem_scholar_abstract,fc_score
0,815vqz,scientific,Signal detected from the first stars in the un...,https://www.nature.com/articles/nature25791,DOI:10.1038/nature25791,Possible interaction between baryons and dark-...,The cosmic radio-frequency spectrum is expecte...,3
1,81ymjh,scientific,Artificial intelligence techniques able to lea...,https://www.nature.com/articles/s41567-018-0048-5,DOI:10.1038/s41567-018-0048-5,Neural-network quantum state tomography,The experimental realization of increasingly c...,1
2,82bpf1,scientific,Thrombin@Fe 3 O 4 nanoparticles for use as a h...,https://www.nature.com/articles/s41598-017-186...,DOI:10.1038/s41598-017-18665-4,Thrombin@Fe3O4 nanoparticles for use as a hemo...,Bleeding remains one of the main causes of pre...,5
3,82dzyr,scientific,Scientists have found a breakthrough technique...,https://www.nature.com/articles/s41557-018-0009-8,DOI:10.1038/s41557-018-0009-8,Control over phase separation and nucleation u...,Control over the nucleation of new phases is h...,2
4,82es30,scientific,Simulating Uneven Trails Provides Insight Into...,https://www.nature.com/articles/s41598-018-210...,DOI:10.1038/s41598-018-21018-4,Challenging human locomotion: stability and mo...,The need to move over uneven terrain is a dail...,4
...,...,...,...,...,...,...,...,...
6813,dph7bi,scientific,Neoceroplatus betaryiensisnov. sp. (Diptera: K...,https://www.nature.com/articles/s41598-019-477...,DOI:10.1038/s41598-019-47753-w,Neoceroplatus betaryiensis nov. sp. (Diptera: ...,Blue shining fungus gnats (Diptera) had been l...,5
6814,dpi8uf,scientific,"In this mouse model of epilepsy, the temperatu...",https://www.nature.com/articles/s41374-019-0335-5,DOI:10.1038/s41374-019-0335-5,Temperature elevation in epileptogenic foci ex...,Physiological brain temperature is an importan...,4
6815,dppdhj,scientific,"Based on brain scans data from 1,774 people wi...",https://www.nature.com/articles/s41467-019-130...,DOI:10.1038/s41467-019-13005-8,Altered structural brain asymmetry in autism s...,Altered structural brain asymmetry in autism s...,4
6816,dpstjb,scientific,Researchers demonstrate generation of 2D &amp;...,https://www.nature.com/articles/s41377-019-0201-7,DOI:10.1038/s41377-019-0201-7,Dielectric metasurfaces for complete and indep...,Metasurfaces are optically thin metamaterials ...,5


In [80]:
df_out.to_csv(f"{abstracts_file_name}_fc_scores.csv", index=False)

In [25]:
df_out

,id,category,title,url,identifier,sem_scholar_title,sem_scholar_abstract,fc_score
6,833kz1,news,"For The First Time Ever, We've Found a Deep-Ea...",https://www.sciencealert.com/super-deep-diamon...,DOI:10.1038/nature25972,CaSiO3 perovskite in diamond indicates the rec...,Laboratory experiments and seismology data hav...,-1
13,5au1po,news,Physicists might have found a way to break the...,http://www.sciencealert.com/physicists-say-the...,DOI:10.1038/srep32815,H-theorem in quantum physics,Remarkable progress of quantum information the...,-1
15,5axort,news,A supercomputer just weighed a prime candidate...,http://www.sciencealert.com/a-supercomputer-ju...,DOI:10.1038/nature20115,Calculation of the axion mass based on high-te...,Unlike the electroweak sector of the standard ...,-1
30,5dr0r8,news,NASA's peer-reviewed EM Drive paper has finall...,http://www.sciencealert.com/it-s-official-nasa...,DOI:10.2514/1.B36120,Measurement of Impulsive Thrust from a Closed ...,A vacuum test campaign evaluating the impulsiv...,-1
38,x46cwu,news,Physicists Broke The Speed of Light With Pulse...,https://www.sciencealert.com/physicists-broke-...,DOI:10.1103/PhysRevLett.126.205001,Slow and Fast Light in Plasma Using Optical Wa...,"Slow and fast light, or large changes in the g...",-1
...,...,...,...,...,...,...,...,...
1788,ykbuis,news,New Hybrid Virus Discovered as Flu And RSV Fus...,https://www.sciencealert.com/new-hybrid-virus-...,DOI:10.1038/s41564-022-01242-5,Coinfection by influenza A virus and respirato...,Interactions between respiratory viruses durin...,-1
1790,ym3mx3,news,A Woman Had Cancer 12 Times by Age 36. Her Gen...,https://www.sciencealert.com/a-woman-had-cance...,DOI:10.1126/sciadv.abq5914,Biallelic germline mutations in MAD1L1 induce ...,Germline mutations leading to aneuploidy are r...,-1
1791,ynsgmc,news,Black Holes in Quantum States Have Surprisingl...,https://www.sciencealert.com/black-holes-in-qu...,DOI:10.1103/PhysRevLett.129.181301,Quantum Signatures of Black Hole Mass Superpos...,We present a new operational framework for stu...,-1
1793,ywn72g,news,"Scientists Created a Black Hole in The Lab, An...",https://www.sciencealert.com/scientists-create...,DOI:10.1103/PhysRevResearch.4.043084,Thermalization by a synthetic horizon,Synthetic horizons in models for quantum matte...,-1
